<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/radvess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
from zipfile import ZipFile as zp
import cv2
import os
import numpy as np
import shutil
from glob import glob
import pickle
from tqdm import tqdm



def save_face(im, count,vid_dir):

  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  faces = face_cascade.detectMultiScale(gray, 1.1, 4)

  x,y,w,h = faces[0]

  face_crop = gray[y:y+h,x:x+w]

  face_crop = cv2.resize(face_crop, (128,128), interpolation = cv2.INTER_AREA)

  cv2.imwrite(vid_dir + "/frame_{d}.jpg".format(d = count), face_crop)


####################################


def load_data(save_pickle = 1):

  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

  for zip_f in tqdm(glob('*.zip')):

      actor_dir = zip_f[:-4]

      if os.path.isdir(actor_dir):
        shutil.rmtree(actor_dir)

      os.mkdir(actor_dir)

      with zp('/content/'+zip_f) as myzip:
        

        for f in tqdm(myzip.infolist()):
          
          vid_dir = os.path.join( actor_dir, f.filename[:-4]) 

          os.mkdir(vid_dir )

          ifile = myzip.extract(f)
          
          vidcap = cv2.VideoCapture(ifile)
             
          n_frames = np.floor((vidcap.get(cv2.CAP_PROP_FRAME_COUNT))).astype(np.int)

          frames_list = list(np.linspace(0,n_frames,60, dtype=np.int))


          success,image = vidcap.read()
          count = 0
          while success:
            if count in frames_list:
                
                save_face(image, count,vid_dir)
                     
            success,image = vidcap.read()
            count += 1
          os.remove(ifile)
  
  
  if save_pickle == 1:
    images2pickle

##############################################

def images2pickle(path=None):

  dirs = glob("/content/*/")

  x = []
  y = []

  for dir in dirs:
    if dir.find('Actor') == -1: continue

    sub_dirs = glob(os.path.join(dir,'*/'))

    for sub_dir in sub_dirs:

      images = np.array(imread_collection(os.path.join(sub_dir, '*.jpg')))

      x.append(images)

      tag = sub_dir.split('/')[3][7]

      y.append(tag)



  x = np.array(x)

  y = np.array(y)

  with open('radvess_data_for_train.pickle', 'wb') as output:
    pickle.dump(x, output)

  with open('radvess_tags_for_train.pickle', 'wb') as output:
    pickle.dump(y, output)


####################################################

def load_pickle(path):

  with open(path, 'rb') as data:
    return pickle.load(data)




In [135]:
load_data()




  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:12<01:48, 12.08s/it]

 20%|██        | 2/10 [00:23<01:36, 12.00s/it]

 30%|███       | 3/10 [00:35<01:24, 12.01s/it]

 40%|████      | 4/10 [00:47<01:12, 12.01s/it]

 50%|█████     | 5/10 [01:00<01:00, 12.08s/it]

 60%|██████    | 6/10 [01:12<00:48, 12.02s/it]

 70%|███████   | 7/10 [01:24<00:36, 12.03s/it]

 80%|████████  | 8/10 [01:36<00:24, 12.01s/it]

 90%|█████████ | 9/10 [01:48<00:11, 11.99s/it]

100%|██████████| 10/10 [01:59<00:00, 11.98s/it]

100%|██████████| 1/1 [01:59<00:00, 119.88s/it]
